The table covidGeoDist contains confirmed cases and deaths in countries, organized by date. This data is sourced from the bigquery public database, in the covid19_ecdc dataset. The original table was titled "covid_19_geographic_distribution_worldwide." The ecdc is the European Centre for Disease Prevention and Control, however, the data contains reports for countries outside of Europe as well.
https://cloud.google.com/blog/products/data-analytics/free-public-datasets-for-covid19


The table googleMobility contains infomation on how mobility trends have changed, sorted by date and country. This change in traffic is quantified by the percent change in traffic on that day in that location from the baseline median value for the corresponding day of the week during the 5 week period January 3 to Febuary 6. Traffic is calculated with data from users that have opted into Location History on their Google account. There are 6 location categories considered for each observation: Grocery & pharmacy, Parks, Transit stations, Retail & recreation, Residential, and Workplaces.
https://www.google.com/covid19/mobility/

Sample of the covidGeoDist table: 

In [36]:
%%bigquery
SELECT * 
FROM covid_staging.covidGeoDist
LIMIT 5

,date,day,month,year,daily_confirmed_cases,daily_deaths,confirmed_cases,deaths,countries_and_territories,geo_id,country_territory_code,pop_data_2018
0,2019-12-31,31,12,2019,0,0,0,0,United_Arab_Emirates,AE,ARE,9630959
1,2019-12-31,31,12,2019,0,0,0,0,Afghanistan,AF,AFG,37172386
2,2019-12-31,31,12,2019,0,0,0,0,Armenia,AM,ARM,2951776
3,2019-12-31,31,12,2019,0,0,0,0,Austria,AT,AUT,8847037
4,2019-12-31,31,12,2019,0,0,0,0,Australia,AU,AUS,24992369


Sample of the googleMobility table:

In [3]:
%%bigquery
SELECT *
FROM covid_staging.googleMobility
LIMIT 5

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,FJ,Fiji,None,None,2020-04-03,-47,-23,-26,-55,-41,29
1,FJ,Fiji,None,None,2020-04-10,-64,-56,-37,-80,-71,35
2,FJ,Fiji,None,None,2020-04-13,-66,-58,-31,-83,-73,32
3,FJ,Fiji,None,None,2020-02-17,-4,-3,-3,-2,9,0
4,FJ,Fiji,None,None,2020-02-19,-2,-4,-2,4,8,0


# I.

How many countries are listed in the covidGeoDist table that are not in the googleMobility table?

In [75]:
%%bigquery
SELECT COUNT(DISTINCT geo_id)
FROM covid_staging.covidGeoDist
WHERE geo_id NOT IN
  (SELECT DISTINCT country_region_code 
   FROM covid_staging.googleMobility)

,f0_
0,81


In [77]:
%%bigquery
SELECT DISTINCT geo_id, countries_and_territories
FROM covid_staging.covidGeoDist
WHERE geo_id NOT IN
  (SELECT DISTINCT country_region_code 
   FROM covid_staging.googleMobility)

,geo_id,countries_and_territories
0,AM,Armenia
1,AZ,Azerbaijan
2,CN,China
3,DZ,Algeria
4,EL,Greece
...,...,...
76,VA,Holy_See
77,VC,Saint_Vincent_and_the_Grenadines
78,VG,British_Virgin_Islands
79,VI,United_States_Virgin_Islands


How many countries are there total in the covidGeoDist table?

In [21]:
%%bigquery
SELECT COUNT(DISTINCT geo_id)
FROM covid_staging.covidGeoDist

,f0_
0,209


So, out of the 209 countries in the covidGeoDist table, 81 of them are not in the googleMobility table

Which countries are in the googleMobility table that are not in the covidGeoDist table?

In [37]:
%%bigquery
SELECT DISTINCT country_region_code, country_region
FROM covid_staging.googleMobility
WHERE country_region_code NOT IN
  (SELECT geo_id
   FROM covid_staging.covidGeoDist)

,country_region_code,country_region
0,GR,Greece
1,RE,Réunion
2,HK,Hong Kong
3,GB,United Kingdom


In [1]:
%%bigquery 
SELECT DISTINCT countries_and_territories, geo_id
FROM covid_staging.covidGeoDist
WHERE countries_and_territories = 'Greece' OR countries_and_territories = 'United_Kingdom'

,countries_and_territories,geo_id
0,Greece,EL
1,United_Kingdom,UK


In [3]:
%%bigquery
SELECT country_region
FROM covid_staging.googleMobility
WHERE country_region = "China"

,country_region


In [23]:
%%bigquery
SELECT COUNT(DISTINCT country_region_code)
FROM covid_staging.googleMobility

,f0_
0,132


Out of the 132 countries in the googleMobility table, 4 of them are not in the covidGeoDist table. These locations are Greece, Réunion, Hong Kong, and the United Kingdom.

Greece and the United Kingdom are in the covidGeoDist table, but have a different country code than in the googleMobility table. These codes can be changed so that they agree between the tables.

Hong Kong is not a country, but a Special Administrative Region of China. However, googleMobility is missing China; thus, Hong Kong can be imputed for the missing China value.

Réunion is also not a country, but a French Department in the Indian Ocean. It will be dropped.

# II.

Which dates are in the googleMobility data that are not in the covidGeoDist data? 

In [55]:
%%bigquery
SELECT g.date
FROM covid_staging.googleMobility g
WHERE g.date NOT IN 
  (SELECT d.date 
  FROM covid_staging.covidGeoDist d)

,date


All dates in the googleMobility table are in the covidGeoDist data

How many dates are in the covidGeoDist data that are not in the googleMobility data?

In [56]:
%%bigquery
SELECT COUNT(DISTINCT d.date) 
FROM covid_staging.covidGeoDist d
WHERE d.date NOT IN
  (SELECT g.date 
  FROM covid_staging.googleMobility g)

,f0_
0,52


In [72]:
%%bigquery
SELECT DISTINCT d.date 
FROM covid_staging.covidGeoDist d
WHERE d.date NOT IN
  (SELECT g.date 
  FROM covid_staging.googleMobility g)
ORDER BY d.date

,date
0,2019-12-31
1,2020-01-01
2,2020-01-02
3,2020-01-03
4,2020-01-04
5,2020-01-05
6,2020-01-06
7,2020-01-07
8,2020-01-08
9,2020-01-09


In [59]:
%%bigquery
SELECT MIN(date), MAX(date)
FROM covid_staging.googleMobility

,f0_,f1_
0,2020-02-15,2020-04-30


So, data collection in both sets took place in a consecutive string of days. However, the GeoDist data collection started before and ended after the googleMobility data collection.

# III.

The countryContinent table has alpha-2 codes for each country with a continent abbreviation from https://dev.maxmind.com/geoip/legacy/codes/country_continent/

In [73]:
%%bigquery
SELECT *
FROM covid_staging.countryContinent 
WHERE continent != '--'
LIMIT 5

,country,continent
0,AO,AF
1,BF,AF
2,BI,AF
3,BJ,AF
4,BW,AF


In [74]:
%%bigquery
SELECT DISTINCT country_region_code, country_region
FROM covid_staging.googleMobility
WHERE country_region_code NOT IN
  (SELECT country
   FROM covid_staging.countryContinent)

,country_region_code,country_region
0,GR,Greece
1,GB,United Kingdom


So, there is a continent for almost every country code in the googleMobility table.

If Hong Kong and Réunion are removed from the googleMobility table, and the country codes for the United Kingdom and Greece are changed to be consistent with the covidGeoDist table, then there will be a continent for every country code in the googleMobility table.

In [66]:
%%bigquery
SELECT DISTINCT continent 
FROM covid_staging.countryContinent 
WHERE continent != '--'

,continent
0,AF
1,AN
2,AS
3,EU
4,NA
5,OC
6,SA
7,continent code


The contCode table is a table that I made in excel. It has each continent code with its continent name.

In [69]:
%%bigquery
SELECT * 
FROM covid_staging.contCode

,code,continent
0,AF,Africa
1,AN,Antartica
2,AS,Asia
3,EU,Europe
4,NA,North America
5,OC,Oceania
6,SA,South America


In [4]:
%%bigquery
SELECT *
FROM covid_staging.countryContinent
WHERE continent NOT IN
  (SELECT code FROM covid_staging.contCode)

,country,continent
0,O1,--


The countryContinent table has an erroneous row, however, every country has a corresponding continent in contCode